In [21]:
# imports
import boto3
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import cm
import os
from shapely.geometry import Point
import sys  # Used for progress bar
from case_study_eval_utils import *
import datetime
import sys
import os

# Import qaqc stage plot functions
sys.path.append(os.path.abspath("../scripts/3_qaqc_data"))
from qaqc_plot import flagged_timeseries_plot, _plot_format_helper, id_flag


# Set AWS credentials
s3 = boto3.resource("s3")
s3_cl = boto3.client("s3")  # for lower-level processes

# Set relative paths to other folders and objects in repository.
BUCKET_NAME = "wecc-historical-wx"
QAQC_DIR = "3_qaqc_wx"
MERGE_DIR = "4_merge_wx"
stations_csv_path = f"s3://{BUCKET_NAME}/{QAQC_DIR}/all_network_stationlist_qaqc.csv"

In [2]:
# set up event start and end dates, and affected counties
event_start_date = "2007-10-05"
event_end_date = "2007-11-30"
event_counties = ['San Diego','Los Angeles','Ventura','Santa Barbara','San Bernardino','Orange','Riverside']

In [3]:
# read in merge station list
stn_list = pd.read_csv(
    "s3://wecc-historical-wx/4_merge_wx/all_network_stationlist_merge.csv"
)

In [4]:
# convert it into geodataframe
stns_gdf = gpd.GeoDataFrame(
    stn_list,
    geometry=gpd.points_from_xy(
        stn_list.longitude, stn_list.latitude, crs="EPSG:4326"
    ),
)

In [7]:
# read in  CA county boundaries shapefile
ca_counties = gpd.read_file(
    "s3://wecc-historical-wx/0_maps/ca_counties/CA_Counties.shp"
)
ca_counties = ca_counties.to_crs(stns_gdf.crs)  # Convert to station CRS

In [8]:
# define the event geometry - we'll focus on once county for now (San Diego - where the largest fires occurred)
event_geom = ca_counties[ca_counties["NAME"] == ("San Diego")]  # .isin(event_counties)]

In [9]:
# filter down to stations that are in the target county
stns_gdf["intersects"] = stns_gdf.intersects(
    event_geom.unary_union
)  # See which stations intersect with the event polygon

event_stns = stns_gdf[stns_gdf["intersects"] == True].reset_index(
    drop=True
)  # Get just those stations, drop the others

In [10]:
# ...AND with start and end date within the event timeframe
event_stns = event_stns[
    (event_stns["start-date"] < event_end_date)
    & (event_stns["end-date"] > event_start_date)
]

In [69]:
def event_plot_multiple_stations(
    sample: list,
    var: str,
    event: str,
    alt_start_date: str | None = None,
    alt_end_date: str | None = None,
    dpi: int | None = None,
):
    """Produces timeseries of variables that have flags placed

    Parameters
    ----------
    df : pd.DataFrame
        stationlist
    var : str
        name of variable
    event : str
        name of case study event
    alt_start_date : str
        date of different event, must be in format "YYYY-MM-DD"
    alt_end_date : str
        date of different event, must be in format "YYYY-MM-DD"
    dpi : int
        figure resolution

    Returns
    -------
    None
    """

    fig, ax = plt.subplots(figsize=(10, 3))
    cmap = cm.get_cmap("rainbow", 10)
    sample = sample.reset_index()

    # plot all observations
    for i in sample.index:
        station = sample.loc[i,"era-id"]
        network = sample.loc[i,"network"]


        url = f"s3://{BUCKET_NAME}/{MERGE_DIR}/{network}/{station}.zarr"

        ds = xr.open_zarr(url)

        df = ds.to_dataframe()
        df = df.reset_index()

        # subset data
        mask = (df["time"] >= "2007-10-05") & (df["time"] <= "2007-11-30")
        df = df.loc[mask]
        if var in df.columns:
            df.plot(
                ax=ax,
                x="time",
                y=var,
                marker=" ",
                ms=4,
                lw=1,
                color = cmap(i),
                alpha=0.5,
                label=station,
            )

            # plot any flags placed by QA/QC
            if len(df[var + "_eraqc"].dropna().unique()) != 0:
                # identify flagged data, can handle multiple flags
                for flag in df[var + "_eraqc"].dropna().unique():
                    flag_name = id_flag(flag)
                    flag_str = 100 * len(df.loc[df[var + "_eraqc"] == flag, var]) / len(df)
                    flag_label = f"{flag_str:.3f}% of data flagged by {flag_name}"

                    flagged_data = df[~df[var + "_eraqc"].isna()]
                    flagged_data.plot(
                        x="time",
                        y=var,
                        ax=ax,
                        marker="o",
                        ms=7,
                        lw=0,
                        mfc="none",
                        color = 'C3',
                        label=flag_label,
                    )

    # plot event timeline
    event_start, event_end = event_info(event, alt_start_date, alt_end_date)
    ax.axvspan(
        event_start,
        event_end,
        color = 'black',
        alpha=0.1,
        label="{}".format(event),
    )
    legend = ax.legend(loc="upper left", prop={"size": 5})

    # plot aesthetics
    ylab, units, miny, maxy = _plot_format_helper(var)
    plt.ylabel(f"{ylab} [{units}]")
    plt.xlabel("")
    stn = df["station"].unique()[0]
    plt.title(
        f"QA/QC event evaluation: {event}",
        fontsize=10,
    )

    return None

In [70]:
# take a sub sample of stations
sub_sample = find_other_events(
    event_stns, event_start_date, event_end_date, 14, 4, return_stn_ids=True
)

Subsetting station record for event duration with 14 day buffer...
4 stations selected for evaluation for comparison!
Stations selected for evaluation:
 ['CWOP_AS488', 'RAWS_MLGC1', 'RAWS_CTGC9', 'CWOP_C4808']


In [76]:
event_plot_multiple_stations(sub_sample, 'sfcWind', 'santa_ana_wind')

plot of ['CWOP_AS488', 'RAWS_MLGC1', 'RAWS_CTGC9', 'CWOP_C4808'] shows potentially anomalous behaviorin surface wind 
generate map of their locations. 


In [77]:
sub_sample

Unnamed: 0      era-id   latitude   longitude  elevation  \
241        6417  CWOP_AS488  33.021330 -116.940170     1540.0   
401       14055  RAWS_MLGC1  32.881217 -116.428803     5737.0   
393       13787  RAWS_CTGC9  33.372222 -117.358889      917.0   
289        8377  CWOP_C4808  32.887600 -117.182900      376.0   

                   start-date                  end-date pulled  \
241 2006-12-06 00:00:00+00:00 2022-01-07 06:07:00+00:00      Y   
401 1999-10-13 00:00:00+00:00 2023-03-07 00:52:00+00:00      Y   
393 1999-12-02 00:00:00+00:00 2011-07-25 17:41:00+00:00      Y   
289 2006-02-23 00:00:00+00:00 2011-03-18 17:45:00+00:00      Y   

                  time_checked network  ... sfcwind_nobs sfcwind_dir_nobs  \
241  2023-04-06 18:02:34+00:00    CWOP  ...       707690           519665   
401  2023-03-24 14:12:34+00:00    RAWS  ...       191382           169222   
393  2023-03-24 14:12:34+00:00    RAWS  ...       288892           287901   
289  2023-04-06 18:02:34+00:00    CWOP  ...       126606           126606   

     rsds_nobs  total_nobs  qaqc                  time_qaqc  merged  \
241         43      707712     Y  2025-05-02 09:51:09+00:00       Y   
401      83370      191789     Y  2025-05-02 07:00:31+00:00       Y   
393          0      291705     Y  2025-05-02 07:00:49+00:00       Y   
289          0      126606     Y  2025-05-01 22:13:57+00:00       Y   

                    time_merge                     geometry  intersects  
241  2025-06-12 19:27:53+00:00  POINT (-116.94017 33.02133)        True  
401  2025-06-12 22:58:17+00:00  POINT (-116.42880 32.88122)        True  
393  2025-06-12 22:42:14+00:00  POINT (-117.35889 33.37222)        True  
289  2025-06-12 19:42:56+00:00  POINT (-117.18290 32.88760)        True  

[4 rows x 35 columns]

In [ ]:
def stn_visualize(stn_list, event_to_eval):
    """
    Produces simple map of station relevant to event boundary.

    Parameters
    ----------
    stn_list : pd.DataFrame
        stationlist
    event_to_eval : str
        name of case study event

    Returns
    -------
    None
    """

    lat_lon_list = []


    for i in stn_list:
        # grab station id info and reproject coords
        #stn = stn_list.loc[stn_list["era-id"] == stn_id]
        lon, lat = stn.longitude.values[0], stn.latitude.values[0]
        x, y = latlon_to_mercator_cartopy(lat, lon)

    # figure set-up
    fig, ax = plt.subplots(subplot_kw={"projection": ccrs.epsg(3857)})
    ax.coastlines()
    ax.add_feature(cf.BORDERS)
    ax.add_feature(cf.STATES, lw=0.5)

    ax.set_extent([lon + 1, lon - 1, lat - 1, lat + 1])

    # Obtain the limits of the plot
    x0, x1, y0, y1 = ax.get_extent()

    # Create a polygon with the limits of the plot
    polygon = Polygon(((x0, y0), (x0, y1), (x1, y1), (x1, y0)))

    # Use only the counties that overlap with the actual plot
    ca_county = gpd.read_file(CENSUS_SHP)
    counties = ca_county[ca_county.overlaps(polygon)]

    # Plot the counties' geometries
    for geometry in counties.geometry:
        ax.add_geometries(
            geometry.boundary,
            crs=ax.projection,
            facecolor="none",
            edgecolor="teal",
            lw=0.5,
        )

    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    ax.plot(lon, lat, "ok", markersize=8, transform=ccrs.PlateCarree(), mfc="none")
    ax.plot(x, y, ".r", markersize=4)
    ax.annotate(f"{stn_id}", xy=(x, y), xytext=(x + 10, y + 10), fontsize=6)
    # station name
    gl = ax.gridlines(
        crs=ccrs.PlateCarree(), draw_labels=["bottom", "left"], ls=":", lw=0.5
    )
    ax.set_title(f"{event_to_eval} evaluation \nat {stn_id}")

    return None